#Verify on UCI dataset
1. Decision Tree Classifier/ Random Forest Classifier

2. Logistic regression

3. SVM

4. XGboost

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output

### Lets start with Importing Libraries and data

In [5]:
import pandas as pd # to import csv and for data manipulation
import matplotlib.pyplot as plt # to plot graph
import numpy as np # for linear algebra
import datetime # to dela with date and time
%matplotlib inline
from sklearn.preprocessing import StandardScaler # for preprocessing the data
from sklearn.ensemble import RandomForestClassifier # Random forest classifier
from sklearn.tree import DecisionTreeClassifier # for Decision Tree classifier
from sklearn.svm import LinearSVC as SVC # for SVM classification
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split # to split the data
from sklearn.cross_validation import KFold # For cross vbalidation
from sklearn.model_selection import GridSearchCV # for tunnig hyper parameter it will use all combination of given parameters
from sklearn.model_selection import RandomizedSearchCV # same for tunning hyper parameter but will use random combinations of parameters
from sklearn.metrics import confusion_matrix,recall_score,precision_recall_curve,auc,roc_curve,roc_auc_score,classification_report
from imblearn.pipeline import make_pipeline
import warnings
warnings.filterwarnings('ignore')

/home/leshare/softwares/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
data = pd.read_csv("../data/real/vehicle.dat",header = 0)

In [7]:
# now let us check in the number of Percentage
data['Class'][data['Class']=='positive'] = 1
data['Class'][data['Class']=='negative'] = 0
Count_Normal_transacation = len(data[data["Class"]==0]) 
Count_Fraud_transacation = len(data[data["Class"]==1]) 
Percentage_of_Normal_transacation = 1.*Count_Normal_transacation/(Count_Normal_transacation+Count_Fraud_transacation)
print("percentage of normal transacation is",Percentage_of_Normal_transacation*100)
Percentage_of_Fraud_transacation= 1.*Count_Fraud_transacation/(Count_Normal_transacation+Count_Fraud_transacation)
print("percentage of fraud transacation",Percentage_of_Fraud_transacation*100)

('percentage of normal transacation is', 74.34988179669031)
('percentage of fraud transacation', 25.65011820330969)


In [8]:
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.metrics import precision_recall_curve, auc, f1_score, precision_recall_fscore_support
from imblearn.metrics import geometric_mean_score

def evaluate_auc_prc(y, pred):
    precision, recall, thresholds = precision_recall_curve(y, pred)
    aucprc = auc(recall, precision)
    #print 'AUPRC:{}'.format(aucprc)
    #plt.title('Precision Recall Curve')
    #plt.plot(precision, recall, 'b',label='AUC = %0.2f'% aucprc)
    #plt.legend(loc='lower right')
    #plt.ylabel('Precision')
    #plt.xlabel('Recall')
    #plt.show()
    return aucprc


def evaluate_f1(y, y_pred, pos_label=1):
    precision, recall, f1, support = precision_recall_fscore_support(y, y_pred, pos_label=1)
    #print classification_report(y, y_pred)
    return f1[1]

aucprc_score = make_scorer(evaluate_auc_prc, greater_is_better=True)

gms = make_scorer(geometric_mean_score, greater_is_better=True)

f1_scorer = make_scorer(evaluate_f1, greater_is_better=True)

def data_split(x, test_size=0.4, seed=1): # preparing data for training and testing as we are going to use different data 
    #again and again so make a function
    x_features= x.ix[:,x.columns != "Class"]
    x_labels=x.ix[:,x.columns=="Class"]
    x_features_train,x_features_test,x_labels_train,x_labels_test = train_test_split(x_features,x_labels,
                                                                                     test_size=test_size, 
                                                                                     random_state=seed)
    x_features_train.index = np.arange(len(x_features_train))
    x_labels_train.index = np.arange(len(x_labels_train))
    x_features_test.index = np.arange(len(x_features_test))
    x_labels_test.index = np.arange(len(x_labels_test))
    print("length of training data")
    print(len(x_features_train))
    print("length of test data")
    print(len(x_features_test))
    return(x_features_train,x_features_test,x_labels_train,x_labels_test)

In [9]:
# before starting we should standridze 
from sklearn.preprocessing import RobustScaler
y = data['Class'].values
X = data.drop(['Class'], axis=1).values
scalar = RobustScaler()
X = scalar.fit_transform(X)
x_train = X
y_train = y.astype('int32')

In [409]:
import pickle
pickle.dump({'x':x_train, 'y':np.eye(2)[y_train.astype('int32')]}, open('../data/real/vehicle/train.pkl', 'wb+'))

In [10]:
from sklearn.neural_network import MLPClassifier as MLP
from sklearn.linear_model import LogisticRegression as LR
from sklearn.svm import LinearSVC as SVM
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.tree import DecisionTreeClassifier as DT
from time import time

In [22]:
# Choose classifer with cross validation
# Supervised Model: LR, SVM (linear), SVM (rbf), DT, KNN, MLP
# Dataset: original
from sklearn.neural_network import MLPClassifier as MLP
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

# LR
# best_c = 0
# best_reward = 0
# for c in [1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]:
#     clf = LR(C=c, random_state=0)
#     reward = np.array(cross_val_score(clf, x_train, y_train, cv=5, n_jobs=5, scoring=gms)).mean()
#     if reward > best_reward:
#         best_reward = reward
#         best_c = c
# print 'best_c for LR:{}'.format(best_c)
# print 'best_reward for LR:{}'.format(best_reward)

# SVM (linear)
# best_c = 0
# best_reward = 0
# for c in [1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]:
#     clf = SVM(C=c, random_state=0)
#     reward = np.array(cross_val_score(clf, x_train, y_train, cv=5, n_jobs=5, scoring=gms)).mean()
#     if reward > best_reward:
#         best_reward = reward
#         best_c = c
# print 'best_c for SVM:{}'.format(best_c)
# print 'best_reward for SVM:{}'.format(best_reward)

# SVM (rbf)
# best_c = 0
# best_reward = 0
# for c in [1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]:
#     clf = SVC(C=c, kernel='rbf', random_state=0)
#     reward = np.array(cross_val_score(clf, x_train, y_train, cv=5, n_jobs=5, scoring=gms)).mean()
#     if reward > best_reward:
#         best_reward = reward
#         best_c = c
# print 'best_c for SVC:{}'.format(best_c)
# print 'best_reward for SVC:{}'.format(best_reward)

# DT 
# best_c = 0
# best_reward = 0
# for c in [1, 2, 3, 4, 5, 6, 7, 8]:
#     clf = DT(max_depth=c, random_state=0)
#     reward = np.array(cross_val_score(clf, x_train, y_train, cv=5, n_jobs=5, scoring=gms)).mean()
#     if reward > best_reward:
#         best_reward = reward
#         best_c = c
# print 'best_c for DT:{}'.format(best_c)
# print 'best_reward DT:{}'.format(best_reward)

# KNN 
# best_c = 0
# best_reward = 0
# for c in [1, 2, 3, 4, 5, 6, 7, 8]:
#     clf = KNN(n_neighbors=c)
#     reward = np.array(cross_val_score(clf, x_train, y_train, cv=5, n_jobs=5, scoring=gms)).mean()
#     if reward > best_reward:
#         best_reward = reward
#         best_c = c
# print 'best_c for KNN:{}'.format(best_c)
# print 'best_reward KNN:{}'.format(best_reward)

# KNN 
best_c = 0
best_reward = 0
for c in [1, 2, 3, 4, 5, 6, 7, 8]:
    clf = MLP(hidden_layer_sizes=[5, c], random_state=0)
    reward = np.array(cross_val_score(clf, x_train, y_train, cv=5, n_jobs=5, scoring=gms)).mean()
    if reward > best_reward:
        best_reward = reward
        best_c = c
print 'best_c for MLP:{}'.format(best_c)
print 'best_reward MLP:{}'.format(best_reward)

best_c for MLP:5
best_reward MLP:0.685563975135


In [9]:
# Choose hyper-parameters with cross validation
# Supervised Model: SVM
# Dataset: original
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC


best_c = 0
best_reward = 0
for c in [1e2]:
    print('c:{}'.format(c))
    clf = SVC(C=c, kernel='rbf', random_state=0)
    reward = np.array(cross_val_score(clf, x_train, y_train, cv=5, n_jobs=5, scoring=gms)).mean()
    if reward > best_reward:
        best_reward = reward
        best_c = c
print 'best_c:{}'.format(best_c)
print 'best_reward:{}'.format(best_reward)
clf = SVC(C=best_c, kernel='rbf', random_state=0)

c:100.0
best_c:100.0
best_reward:0.803499737701


In [10]:
# Performance on original dataset with the chosen model.
clf.fit(x_train, y_train)
preds = clf.predict(x_train)
reward = geometric_mean_score(y_train, preds)
print 'reward on original dataset:{}'.format(reward)

reward on original dataset:0.935698711428


In [423]:
# Choose under-sampling ratio for different under-sampling methods
# Supervised Model: SVM(C=1e2, kernel='rbf', random_state=0)
# Under sampling methods: Random, ENN, Cluster, TomekLinks, ALLKNN
from imblearn.under_sampling import (NearMiss, RandomUnderSampler, EditedNearestNeighbours, 
                                     CondensedNearestNeighbour, ClusterCentroids, TomekLinks,
                                     RepeatedEditedNearestNeighbours, AllKNN)
from imblearn.pipeline import make_pipeline
from collections import Counter


train_x = x_train
train_y = y_train
pos_num = (train_y == 1).sum()
neg_num = (train_y == 0).sum()


for sampler, name, time in zip([RandomUnderSampler, EditedNearestNeighbours, ClusterCentroids, TomekLinks, AllKNN],
                         ['Random', 'ENN', 'Cluster', 'TomekLinks', 'ALLKNN'], [50, 1, 1, 1, 1]):
    max_i = 1
    best_reward = -1
    for i in np.arange(1, 1.0*neg_num/pos_num, 0.2):
        sample = sampler(ratio={0:int(i*pos_num)})
        train_x_s, train_y_s = sample.fit_sample(train_x, train_y)
        clf.fit(train_x_s, train_y_s)
        preds = clf.predict(train_x)
        reward = geometric_mean_score(train_y, preds)
        if best_reward < reward:
            best_reward = reward    
            max_i = i
    print 'best under-sampling ratio with {}:{}'.format(name, max_i)
    
    best_reward = 0
    for i in np.arange(time):
        sample = sampler(ratio={0:int(max_i*pos_num)})
        train_x_s, train_y_s = sample.fit_sample(train_x, train_y)
        clf.fit(train_x_s, train_y_s)
        preds = clf.predict(train_x)
        reward = geometric_mean_score(train_y, preds)
        if best_reward < reward:
            best_reward = reward    
    print 'best reward with {}:{}'.format(name, best_reward)

best under-sampling ratio with Random:2.0
best reward with Random:0.949336798322
best under-sampling ratio with ENN:1.0
best reward with ENN:0.877197445112
best under-sampling ratio with Cluster:1.4
best reward with Cluster:0.936653466352
best under-sampling ratio with TomekLinks:1.0
best reward with TomekLinks:0.938146254243
best under-sampling ratio with ALLKNN:1.0
best reward with ALLKNN:0.857956310197


In [33]:
# Choose under-sampling ratio for different under-sampling methods
# Supervised Model: SVM(C=1e2, kernel='rbf', random_state=0)
# Under sampling methods: Random, ENN, Cluster, TomekLinks, ALLKNN
from imblearn.under_sampling import (NearMiss, RandomUnderSampler, EditedNearestNeighbours, 
                                     CondensedNearestNeighbour, ClusterCentroids, TomekLinks,
                                     RepeatedEditedNearestNeighbours, AllKNN)
from imblearn.pipeline import make_pipeline
from collections import Counter

clf = SVC(C=1e2, kernel='rbf', random_state=0)
train_x = x_train
train_y = y_train
pos_num = (train_y == 1).sum()
neg_num = (train_y == 0).sum()


for sampler, name, time in zip([RandomUnderSampler],
                         ['Random'], [50, 1, 1, 1, 1]):
    max_i = 1
    best_reward = -1
    for i in np.arange(1, 1.0*neg_num/pos_num, 0.2):
        sample = sampler(ratio={0:int(i*pos_num)})
        train_x_s, train_y_s = sample.fit_sample(train_x, train_y)
        clf.fit(train_x_s, train_y_s)
        preds = clf.predict(train_x)
        reward = geometric_mean_score(train_y, preds)
        if best_reward < reward:
            best_reward = reward    
            max_i = i
    print 'best under-sampling ratio for {}:{}'.format(name, max_i)
    
    best_reward = 0
    for i in np.arange(time):
        sample = sampler(ratio={0:int(max_i*pos_num)})
        train_x_s, train_y_s = sample.fit_sample(train_x, train_y)
        clf.fit(train_x_s, train_y_s)
        preds = clf.predict(train_x)
        reward = geometric_mean_score(train_y, preds)
        if best_reward < reward:
            best_reward = reward    
    print 'best reward with {}:{}'.format(name, best_reward)

best under-sampling ratio for Random:2.2
best reward with Random:0.951518307469


In [46]:
# Perform EasyEnsemble and BalanceCascade
from imblearn.ensemble import EasyEnsemble, BalanceCascade

clf = SVC(C=1e2, kernel='rbf', random_state=0)
train_x = x_train
train_y = y_train
pos_num = (train_y == 1).sum()
neg_num = (train_y == 0).sum()

for sampler, name in zip([EasyEnsemble],
                         ['EasyEnsemble']):
    max_i = 1
    best_reward = -1
    for i in [2.0]:
        sample = sampler(ratio={0:int(i*pos_num)}, replacement=False, n_subsets=10)
        train_x_s, train_y_s = sample.fit_sample(x_train, y_train)
        preds = None
        print(len(train_x_s))
        for x, y in zip(train_x_s, train_y_s):
            clf.fit(x, y)
            if preds is None:
                preds = clf.predict(train_x)
            else:
                preds += clf.predict(train_x)
        print(preds[:10])
        preds = (preds >= ((len(train_x_s)+1)/2)).astype('int32')
        reward = geometric_mean_score(train_y, preds)
        if best_reward < reward:
            best_reward = reward    
            max_i = i
    print 'best under-sampling ratio for {}:{}'.format(name, max_i)
    print 'best reward for {}:{}'.format(name, best_reward)


10
[ 0  0 10  0  0  0  0  0  0 10]
best under-sampling ratio for EasyEnsemble:2.0
best reward for EasyEnsemble:0.952753304063


In [426]:
# Choose over-sampling ratio for different orver-sampling methods
# Supervised Model: SVM(C=1e2, kernel='rbf', random_state=0)
# Over sampling methods: Random, SMOTE, ADASYN
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.pipeline import make_pipeline
from collections import Counter

for sampler, name, time in zip([RandomOverSampler, SMOTE, ADASYN],
                         ['Random', 'SMOTE', 'ADASYN'], [50, 10, 10]):
    max_i = 1
    best_reward = -1
    for i in np.arange(1, 1.0*neg_num/pos_num, 0.5):
        sample = sampler(ratio={1:int(i*pos_num)})
        train_x_s, train_y_s = sample.fit_sample(train_x, train_y)
        clf.fit(train_x_s, train_y_s)
        preds = clf.predict(train_x)
        reward = geometric_mean_score(train_y, preds)
        if best_reward < reward:
            best_reward = reward    
            max_i = i
    print 'best over-sampling ratio with {}:{}'.format(name, max_i)
    
    best_reward = 0
    bset_set = None
    for i in np.arange(time):
        sample = sampler(ratio={1:int(max_i*pos_num)})
        train_x_s, train_y_s = sample.fit_sample(train_x, train_y)
        clf.fit(train_x_s, train_y_s)
        preds = clf.predict(train_x)
        reward = geometric_mean_score(train_y, preds)
        if best_reward < reward:
            best_reward = reward  
            bset_set = (train_x_s, train_y_s)
    #pickle.dump({'x':bset_set[0], 'y':np.eye(2)[bset_set[1].astype('int32')]}, open('../data/real/vehicle/train_{}.pkl'.format(name), 'wb+'))
    print 'best reward with {}:{}'.format(name, best_reward)

best over-sampling ratio with Random:2.5
best reward with Random:0.965213717201
best over-sampling ratio with SMOTE:2.5
best reward with SMOTE:0.966268210363
best over-sampling ratio with ADASYN:2.5
best reward with ADASYN:0.966859436584


In [425]:
import pickle
pickle.dump({'x':train_x, 'y':np.eye(2)[train_y.astype('int32')]}, open('../data/real/vehicle/train.pkl', 'wb+'))

In [ ]:
# Test performance of over-sampling methods with chosen ratio
# Supervised Model: SVM(C=1e2, kernel='rbf', random_state=0)
# Over sampling methods: Random, SMOTE, ADASYN
best_reward = 0
sampler = ADASYN(ratio={0:int(max_i*pos_num)})

for i in np.range(50):
    pos_num = (train_y == 1).sum()
    neg_num = (train_y == 0).sum()
    train_x_s, train_y_s = sampler.fit_sample(train_x, train_y)
    clf.fit(train_x_s, train_y_s)
    preds = clf.predict(train_x)
    reward = geometric_mean_score(train_y, preds)
    if best_reward < reward:
        best_reward = reward    
print 'best reward:{}'.format(best_reward)

In [18]:
# Ensemble
# Supervised Model: SVM(C=1e2, kernel='rbf', random_state=0)
# Ensemble methods: EasyEnsemble, BalanceCasecade
from imblearn.ensemble import BalanceCascade, EasyEnsemble
from imblearn.pipeline import make_pipeline
from collections import Counter


train_x = x_train
train_y = y_train
pos_num = (train_y == 1).sum()
neg_num = (train_y == 0).sum()


for sampler, name, time in zip([EasyEnsemble, BalanceCascade],
                         ['EasyEnsemble', 'BalanceCascade'], [50, 1, 1, 1, 1]):
    
    sample = sampler(n_subsets=10)
    train_x_s, train_y_s = sample.fit_sample(train_x, train_y)
    print(train_x_s.shape)
    print(train_y_s.shape)
    preds = np.zeros(len(train_y))
    for i in range(len(train_x_s)):
        clf.fit(train_x_s[i], train_y_s[i])
        preds += clf.predict(train_x)
    preds = (preds > 5).astype('int32')
    reward = geometric_mean_score(train_y, preds)
    print 'best reward with {}:{}'.format(name, reward)

(10, 434, 18)
(10, 434)
best reward with EasyEnsemble:0.935945319289
(4, 434, 18)
(4, 434)
best reward with BalanceCascade:0.0


### MLP for multiply operation

In [347]:
x = np.random.rand(100000, 2)
y = x[:, 0] * x[:, 1]
np.random.seed = 1234
x_test = np.random.rand(20000, 2)
y_test = x_test[:, 0] * x_test[:, 1]
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
rg = MLPRegressor(hidden_layer_sizes=(100, ), activation='tanh')
preds = rg.fit(x, y).predict(x)
mse = mean_squared_error(y, preds)
print 'train mse:{}'.format(mse)
preds = rg.predict(x_test)
mse = mean_squared_error(y_test, preds)
error = (y_test-preds)
print mse

train mse:2.09370660105e-05
2.13083930475e-05


In [348]:
error = np.abs(error)

In [350]:
print error.max()
print error.min()
print error.mean()

0.0298732262179
6.23440807701e-07
0.00344771853463
